In [2]:
import time
from time import process_time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules
#import seaborn as sns
#import warnings
#from PyARMViz import PyARMViz


In [3]:
order = pd.read_csv('order_products_train.csv')
product = pd.read_csv('products.csv',encoding = 'gbk')
aisles = pd.read_csv('aisles.csv')
department = pd.read_csv('departments.csv')

In [7]:
order.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [9]:
#product.shape
order.groupby(['product_id'])['order_id'].count()

product_id
1          76
2           4
3           6
4          22
5           1
         ... 
49682       5
49683    2413
49686       7
49687       1
49688       4
Name: order_id, Length: 39123, dtype: int64

# 数据整理

In [4]:
#将order中的productid与name匹配
r=order.copy()
for i in range(len(r)):
    a=int(r.loc[i,'product_id'])
    r.loc[i,'product_name']=product['product_name'][a-1]
r.head()
x=r.loc[:,['order_id','product_name']]
x.to_csv('order_products_name.csv', index=False)

In [7]:
product.iloc[]

,order_id,product_id,add_to_cart_order,reordered,product_name
0,1,49302,1,1,Bulgarian Yogurt
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese
2,1,10246,3,0,Organic Celery Hearts
3,1,49683,4,0,Cucumber Kirby
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil


In [5]:
#整理为每个订单所包含什么product的形式
df=order.iloc[:,:2]
a=0
dataset=[]
for i in range(len(df)-1):
    if df['order_id'][i] != df['order_id'][i+1]:
        b=list(df['product_id'][a:i])
        dataset.append(b)
        a=i+1
dataset[:4]

[[49302, 11109, 10246, 49683, 43633, 13176, 47209],
 [39612, 19660, 49235, 43086, 46620, 34497, 48679],
 [11913, 18159, 4461, 21616, 23622, 32433, 28842, 42625],
 [20574, 30391, 40706, 25610, 27966, 24489]]

In [21]:
data.copy()
data.iloc[:,1] = data.iloc[:,1].apply(int)
data.iloc[:,0] = data.iloc[:,0].apply(int)
a = data.iloc[:,[0,1]].groupby(by = ['order_id'])['product_id'].apply(list)
a = a.to_frame()
a.head()

,product_id
order_id,
1,"[49302, 11109, 10246, 49683, 43633, 13176, 472..."
36,"[39612, 19660, 49235, 43086, 46620, 34497, 486..."
38,"[11913, 18159, 4461, 21616, 23622, 32433, 2884..."
96,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]"
98,"[8859, 19731, 43654, 13176, 4357, 37664, 34065..."


In [19]:
#将dataset转化为one-hot encoded pandas DataFrame；不使用稀疏格式
te = TransactionEncoder()
#te_ary = te.fit(dataset).transform(dataset)
origin_df = pd.DataFrame(pd.DataFrame(te.fit(a['product_id']).transform(a['product_id']),columns=te.columns_))


#将dataset转化为one-hot encoded pandas DataFrame；使用稀疏格式
te = TransactionEncoder()
oht_ary = te.fit(a['product_id']).transform(a['product_id'], sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
sparse_df.columns = [str(i) for i in sparse_df.columns]

In [22]:
origin_df.head()

,1,2,3,4,5,7,8,9,10,11,...,49677,49678,49679,49680,49681,49682,49683,49686,49687,49688
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# 关联规则挖掘 apriori和fpgrowth算法

In [28]:
#print("method = apriori, sparse = T")
supportLevels=[0.05, 0.01, 0.005]

In [44]:
def apriori_o(n):
    time1=[]
    for i in range(5):
        t1=time.time()
        #计算频繁项集
        frequent_itemsets = apriori(origin_df, min_support=supportLevels[n], use_colnames=True,low_memory = False)
        t2=time.time()
        time1.append(t2-t1)
    mean=sum(time1)/5
    print("min_support=",supportLevels[n],": 运行时间是: {:9.9}s".format(mean))
    return mean

In [45]:
def apriori_s(n):
    time1=[]
    for i in range(5):
        t1=time.time()
        #计算频繁项集
        frequent_itemsets = apriori(sparse_df, min_support=supportLevels[n], use_colnames=True,low_memory = False)
        t2=time.time()
        time1.append(t2-t1)
    mean=sum(time1)/5
    print("min_support=",supportLevels[n],": 运行时间是: {:9.9}s".format(mean))
    return mean

In [46]:
def fpgrowth_o(n):
    time1=[]
    for i in range(5):
        t1=time.time()
        #计算频繁项集
        frequent_itemsets = fpgrowth(origin_df, min_support=supportLevels[n], use_colnames=True)
        t2=time.time()
        time1.append(t2-t1)
    mean=sum(time1)/5
    print("min_support=",supportLevels[n],": 运行时间是: {:9.9}s".format(mean))
    return mean

In [47]:
def fpgrowth_s(n):
    time1=[]
    for i in range(5):
        t1=time.time()
        #计算频繁项集
        frequent_itemsets = fpgrowth(sparse_df, min_support=supportLevels[n], use_colnames=True)
        t2=time.time()
        time1.append(t2-t1)
    mean=sum(time1)/5
    print("min_support=",supportLevels[n],": 运行时间是{:9.9}s".format(mean))
    return mean

In [50]:
apro=[]
for j in range(3):
    a=apriori_o(j)
    apro.append(a)

min_support= 0.05 : 运行时间是: 5.35183735s
min_support= 0.01 : 运行时间是: 26.304055s
min_support= 0.005 : 运行时间是: 145.945445s


In [49]:
aprs=[]
for j in range(3):
    a=apriori_s(j)
    aprs.append(a)

min_support= 0.05 : 运行时间是: 0.561134577s
min_support= 0.01 : 运行时间是: 4.21549654s
min_support= 0.005 : 运行时间是: 26.2869054s


In [48]:
fpgo=[]
for j in range(3):
    a=fpgrowth_o(j)
    fpgo.append(a)

min_support= 0.05 : 运行时间是: 7.15972738s
min_support= 0.01 : 运行时间是: 7.49951353s
min_support= 0.005 : 运行时间是: 8.62701316s


In [42]:
fpgs=[]
for j in range(3):
    a=fpgrowth_s(j)
    fpgs.append(a)

min_support= 0.05 : 运行时间是0.926059294s
min_support= 0.01 : 运行时间是 1.878476s
min_support= 0.005 : 运行时间是2.91315603s
